In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import torch
import torch.nn as nn
from torchvision.datasets import MNIST 
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch_directml
from PIL import Image



In [ ]:
device = torch_directml.device()
print(device)

privateuseone:0


In [ ]:
transform= transforms.ToTensor()


In [ ]:

train_data=MNIST(
    root='data',
    train=True,
    download=True,
    transform=transform
)

test_data=MNIST(
    root='data',
    train=False,
    download=True,
    transform=transform
)
train_loader=DataLoader(
    train_data,
    batch_size=64,
    shuffle=True
)
test_loader=DataLoader(
    test_data,
    batch_size=64,
    shuffle=False
)

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class DigitCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


In [ ]:
model = DigitCNN().to(device)

loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
epochs=5


In [ ]:
for i in range(epochs):
    model.train()
    total_loss=0
    for images,labels in train_loader:
        images=images.view(images.size(0),-1).to(device)
        labels=labels.to(device)
        output=model(images)

        loss=loss_fn(output,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss=loss.item()
   
    print(f'epoch : {i+1} , loss : {total_loss}')

epoch : 1 , loss : 0.09666796773672104
epoch : 2 , loss : 0.08090122044086456
epoch : 3 , loss : 0.3469885289669037
epoch : 4 , loss : 0.20355433225631714
epoch : 5 , loss : 0.04259189963340759


In [ ]:
model.eval()
correct=0
total=0
with torch.no_grad():
    for images , labels in train_loader:
        images=images.view(images.size(0),-1).to(device)
        output=model(images).to(device)

        predictions=output.argmax(dim=1)
        correct+=(predictions==(labels.to(device))).sum().item()
        total+=labels.size(0)
    accuracy=100*(correct/total)
    print('Accuracy : ',round(accuracy,2))

Accuracy :  98.31


In [ ]:

transform=transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((28,28)),
    transforms.ToTensor(),
])

image=Image.open('OIP5.webp')
image=transform(image).to(device)
image=image.unsqueeze(0)

with torch.no_grad():
    output=model(image)
    pred=torch.argmax(output,dim=1)

print('Predicted Digit : ',pred.item())

Predicted Digit :  3


In [ ]:
torch.save(model.state_dict(),'DigitModel.pth')

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.network=nn.Sequential(
            nn.Linear(28*28,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,10)
        )
    def forward(self,x):
        x=x.view(x.size(0),-1)
        return self.network(x)

In [ ]:
model=MyModel()
model.load_state_dict(torch.load('DigitModel.pth',map_location='cpu'))
model.eval()

transform=transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((28,28)),
    transforms.ToTensor(),
])

image=Image.open('3.webp')
image=transform(image)
image=image.unsqueeze(0)

with torch.no_grad():
    output=model(image)
    pred=torch.argmax(output,dim=1)

print('Predicted Digit : ',pred.item())

Predicted Digit :  3


C:\Users\mushe\AppData\Local\Temp\ipykernel_8292\328916337.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('DigitModel.pth',map_location